## Azure OpenAI GPT-V Target Demo
This notebook demonstrates how to use the Azure OpenAI GPT-V target to accept multimodal input (text+image) and generate text output.

In [1]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.


from pyrit.models import PromptRequestPiece, PromptRequestResponse
from pyrit.prompt_target import AzureOpenAIGPTVChatTarget
from pyrit.common import default_values
import pathlib
from pyrit.common.path import HOME_PATH
import uuid

default_values.load_default_env()
test_conversation_id = str(uuid.uuid4())

# use the image from our docs
image_path = pathlib.Path(HOME_PATH) / "assets" / "pyrit_architecture.png"

request_pieces = [
    PromptRequestPiece(
        role="user",
        conversation_id=test_conversation_id,
        original_value="Describe this picture:",
        original_value_data_type="text",
        converted_value_data_type="text",
    ),
    PromptRequestPiece(
        role="user",
        conversation_id=test_conversation_id,
        original_value=str(image_path),
        original_value_data_type="image_path",
        converted_value_data_type="image_path",
    ),
]

In [2]:
prompt_request_response = PromptRequestResponse(request_pieces=request_pieces)

In [3]:
with AzureOpenAIGPTVChatTarget() as azure_openai_chat_target:
    resp = await azure_openai_chat_target.send_prompt_async(prompt_request=prompt_request_response)  # type: ignore
    print(resp)

None: assistant: The picture shows a chart with the title "PyRIT Components" at the top. The chart is divided into two columns, with "Interface" on the left and "Implementation" on the right. Underneath the columns, there are five rows, each labeled with a different component of PyRIT. 

The first row is labeled "Target" and includes two types of implementation: "Local," which specifies local model (e.g., ONNX), and "Remote," which mentions API or web app.

The second row is labeled "Datasets" and includes two types of implementation as well: "Static," which specifies prompts, and "Dynamic," which mentions prompt templates.

The third row is labeled "Scoring Engine" and includes only one type of implementation: PyRIT itself for self-evaluation and API for existing content classifiers.

The fourth row is labeled "Attack Strategy" and also has two implementations: "Single Turn," which uses static prompts, and "Multi-Turn," which involves multiple conversations using prompt templates.

Th